In [1]:
%pip install -U langchain langchain-community langchain-google-genai langchain_experimental mysql-connector-python pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Google Gemini LLM & API key setup

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# keep API key in code for now (your choice)
api_key = ""

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # safer & free tier friendly
    google_api_key="",
    temperature=0.2
)

# test
response = llm.invoke("Write a short sales pitch for a T-shirt brand.")
print(response.content)


Tired of boring tees?  [Brand Name] offers premium quality, unique designs that speak to your personality.  From witty slogans to eye-catching graphics, we've got the perfect shirt to express your style.  Check out our collection and find your new favorite tee – comfort and confidence, guaranteed.


#### Connect with database and ask some basic questions

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

import urllib.parse

db_user = "root"
db_password = urllib.parse.quote_plus("")  # Encode special chars
db_host = "127.0.0.1"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")

C:\Users\Kapil\AppData\Local\Temp\ipykernel_7848\1656595225.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:Question: How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5;
SQLResult: [(32,)]
Answer:Question: How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5;
SQLResult: [(32,)]
Answer:Answer: We do not have any t-shirts from Nike that are extra small and white.  The query returned an empty set.
> Finished chain.
Answer: We do not have any t-shirts from Nike that are extra small and white.  The query returned an empty set.
> Finished chain.


Above is the correct answer 👍🏼

In [5]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

C:\Users\Kapil\AppData\Local\Temp\ipykernel_7848\521761031.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:```sql
SELECT
  SUM(`price` * `stock_quantity`) AS total_price
FROM `t_shirts`
WHERE
  `size` = 'S';
``````sql
SELECT
  SUM(`price` * `stock_quantity`) AS total_price
FROM `t_shirts`
WHERE
  `size` = 'S';
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT\n  SUM(`price` * `stock_quantity`) AS total_price\nFROM `t_shirts`\nW' at line 1")
[SQL: ```sql
SELECT
  SUM(`price` * `stock_quantity`) AS total_price
FROM `t_shirts`
WHERE
  `size` = 'S';
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names. We need to tell it somehow that price is price per unit and the actual query should be,

SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [6]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:Question: SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('24327'),)]
Answer:Question: SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('24327'),)]
Answer:Answer: 363
> Finished chain.
Answer: 363
> Finished chain.


we will use qns2 value later on in this notebook. So hold on for now and let's check another query

In [7]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:```sql
SELECT 
    SUM((`price` * (1 - (`pct_discount` / 100))) * `stock_quantity`) AS total_revenue
FROM
    `t_shirts`
INNER JOIN
    `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE
    `brand` = 'Levi';
``````sql
SELECT 
    SUM((`price` * (1 - (`pct_discount` / 100))) * `stock_quantity`) AS total_revenue
FROM
    `t_shirts`
INNER JOIN
    `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE
    `brand` = 'Levi';
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT \n    SUM((`price` * (1 - (`pct_discount` / 100))) * `stock_quantit' at line 1")
[SQL: ```sql
SELECT 
    SUM((`price` * (1 - (`pct_discount` / 100))) * `stock_quantity`) AS total_revenue
FROM
    `t_shirts`
INNER JOIN
    `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE
    `brand` = 'Levi';
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

Above, it returned a wrong query which generated an error during query execution. It thinks discount table would have start and end date which is normally true but in our table there is no start or end date column. One thing we can do here is run the query directly.

In [14]:

sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:```sql
SELECT
  SUM(a.total_amount * ((100 - COALESCE(d.`pct_discount`, 0)) / 100)) AS total_revenue
FROM (
  SELECT
    SUM(`price` * `stock_quantity`) AS total_amount,
    `t_shirt_id`
  FROM `t_shirts`
  WHERE
    `brand` = 'Levi'
  GROUP BY
    `t_shirt_id`
) a
LEFT JOIN `discounts` d
  ON a.`t_shirt_id` = d.`t_shirt_id`;
``````sql
SELECT
  SUM(a.total_amount * ((100 - COALESCE(d.`pct_discount`, 0)) / 100)) AS total_revenue
FROM (
  SELECT
    SUM(`price` * `stock_quantity`) AS total_amount,
    `t_shirt_id`
  FROM `t_shirts`
  WHERE
    `brand` = 'Levi'
  GROUP BY
    `t_shirt_id`
) a
LEFT JOIN `discounts` d
  ON a.`t_shirt_id` = d.`t_shirt_id`;
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT\n  SUM(a.total_amount * ((100 - COALESCE(d.`pct_discount`, 0)) / 10' at line 1")
[SQL: ```sql
SELECT
  SUM(a.total_amount * ((100 - COALESCE(d.`pct_discount`, 0)) / 100)) AS total_revenue
FROM (
  SELECT
    SUM(`price` * `stock_quantity`) AS total_amount,
    `t_shirt_id`
  FROM `t_shirts`
  WHERE
    `brand` = 'Levi'
  GROUP BY
    `t_shirt_id`
) a
LEFT JOIN `discounts` d
  ON a.`t_shirt_id` = d.`t_shirt_id`;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

It produced a correct answer when explicit query was given. The total revenue without discounts would be higher. The total discount applied reduces the final revenue. Hence we get the post-discount revenue.

Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [9]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('23584'),)]
Answer:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('23584'),)]
Answer:Answer: 3150
> Finished chain.
Answer: 3150
> Finished chain.


In [ ]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")

Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [10]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('200'),)]
Answer:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('200'),)]
Answer:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
> Finished chain.
Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
> Fini

#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [15]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "32"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "24327"},
    {'Question': "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "22309.55"} ,
     {'Question' : "If we have to sell all the Levi's T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "23584"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "200"
     }
]

In [16]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [17]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplate using input variables

In [25]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [26]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [27]:
# Create a proper few-shot prompt using FewShotPromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

# Use the standard approach with example selector
few_shot_prompt = FewShotPromptTemplate(
    examples=few_shots,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [28]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [29]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('200'),)]
Answer:SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('200'),)]
Answer:Answer: 71
> Finished chain.
Answer: 71
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': 'Answer: 71'}

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [30]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('6488'),)]
Answer:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('6488'),)]
Answer:Answer: 6488
> Finished chain.
Answer: 6488
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': 'Answer: 6488'}

In [31]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23594.500000'),)]
Answer:SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23594.500000'),)]
Answer:23594.50
> Finished chain.
23594.50
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '23594.50'}

In [32]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25509.800000'),)]
Answer:SQLQuery: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25509.800000'),)]
Answer:Answer: 25509.8
> Finished chain.
Answer: 25509.8
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'Answer: 25509.8'}

In [33]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Van Huesen'
SQLResult: [(Decimal('27264'),)]
Answer:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Van Huesen'
SQLResult: [(Decimal('27264'),)]
Answer:27264
> Finished chain.
27264
> Finished chain.


'27264'